In [1]:
# importing pyspark 
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
# creating session in pyspark 
spark = SparkSession.builder \
    .appName("GoogleDataset") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .config("spark.sql.execution.pythonUDF.arrow.enabled", "true") \
    .getOrCreate()

In [4]:
# loading dataset in RDD
df = spark.read.csv("E:/DBDA Project/Google-Playstore.csv", header=True, inferSchema=True)

In [5]:
# printing first 20 records 
df.show()

+----------------------------------+--------------------+-----------------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+------------+--------------+--------------------+------------+----------------+--------------+-------------------+
|                          App Name|              App Id|         Category|Rating|Rating Count|Installs|Minimum Installs|Maximum Installs|Free|Price|Currency|Size|Minimum Android|        Developer Id|   Developer Website|     Developer Email|    Released|Last Updated|Content Rating|      Privacy Policy|Ad Supported|In App Purchases|Editors Choice|       Scraped Time|
+----------------------------------+--------------------+-----------------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+--------------------+--------------------+--------------------+------------+-----

In [6]:
# Count the number of elements in the RDD
num_rows = df.count()
print("Number of rows",num_rows)


Number of rows 2312944


In [7]:
# count the number of columns 
sample_row = df.first()
num_columns = len(sample_row)
print("Number of columns:", num_columns)

Number of columns: 24


In [8]:
# schema of the table
df.printSchema()

root
 |-- App Name: string (nullable = true)
 |-- App Id: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Rating Count: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Minimum Installs: string (nullable = true)
 |-- Maximum Installs: string (nullable = true)
 |-- Free: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Minimum Android: string (nullable = true)
 |-- Developer Id: string (nullable = true)
 |-- Developer Website: string (nullable = true)
 |-- Developer Email: string (nullable = true)
 |-- Released: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Privacy Policy: string (nullable = true)
 |-- Ad Supported: string (nullable = true)
 |-- In App Purchases: string (nullable = true)
 |-- Editors Choice: string (nullable = true)
 |-- Scra

In [9]:
# printing null values in the table 
from pyspark.sql.functions import col, sum as pyspark_sum

# Replace 'df' with your DataFrame
null_counts = df.select([pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Display the null counts
null_counts.show()

+--------+------+--------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+------------+-----------------+---------------+--------+------------+--------------+--------------+------------+----------------+--------------+------------+
|App Name|App Id|Category|Rating|Rating Count|Installs|Minimum Installs|Maximum Installs|Free|Price|Currency|Size|Minimum Android|Developer Id|Developer Website|Developer Email|Released|Last Updated|Content Rating|Privacy Policy|Ad Supported|In App Purchases|Editors Choice|Scraped Time|
+--------+------+--------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+------------+-----------------+---------------+--------+------------+--------------+--------------+------------+----------------+--------------+------------+
|       0|     0|       0| 22883|       22883|     107|             107|               0|   0|    0|     135| 196|           6530|      

In [10]:
# values count in the carwgory column 
from pyspark.sql.functions import count

# Replace 'df' with your DataFrame
category_counts = df.groupBy('Category').agg(count('*').alias('count'))

# Display the counts
category_counts.show(100)

+-----------------------+------+
|               Category| count|
+-----------------------+------+
|          Music & Audio|154905|
|              Education|241086|
|                 Trivia| 11795|
|        Auto & Vehicles| 18280|
|          Entertainment|138271|
|              Adventure| 23203|
|   com.free074a81ba9...|     1|
|                 Arcade| 53792|
|   net.cleverbit.Mic...|     1|
|                 Sports| 47483|
|         Travel & Local| 67288|
|           Food & Drink| 73926|
|           Role Playing| 10034|
|                Finance| 65465|
|        Personalization| 89210|
|                 Racing| 10361|
|                  Tools|143987|
|            Educational| 21307|
|                 Comics|  2862|
|                 Social| 44733|
|       Libraries & Demo|  5198|
|               Shopping| 75253|
|       Health & Fitness| 83510|
|           House & Home| 14369|
|           Productivity| 79695|
|                   Card|  8179|
|              Samurai]"|     1|
|         

In [11]:
# Import necessary functions
from pyspark.sql.functions import col, countDistinct

# Calculate the count of distinct values for each column
distinct_counts = df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns))

# Display the count of distinct values
distinct_counts.show()


+--------+-------+--------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+------------+-----------------+---------------+--------+------------+--------------+--------------+------------+----------------+--------------+------------+
|App Name| App Id|Category|Rating|Rating Count|Installs|Minimum Installs|Maximum Installs|Free|Price|Currency|Size|Minimum Android|Developer Id|Developer Website|Developer Email|Released|Last Updated|Content Rating|Privacy Policy|Ad Supported|In App Purchases|Editors Choice|Scraped Time|
+--------+-------+--------+------+------------+--------+----------------+----------------+----+-----+--------+----+---------------+------------+-----------------+---------------+--------+------------+--------------+--------------+------------+----------------+--------------+------------+
| 2177946|2312944|      78|    66|       38497|      41|              38|          251569|  28| 1072|      19|1660|            174|  

In [12]:
# Specify the columns to drop
columns_to_drop = ['App Id', 'Developer Id', 'Developer Website', 'Developer Email', 'Privacy Policy', 'Editors Choice', 'Minimum Android', 'Scraped Time']

# Drop the specified columns
df = df.drop(*columns_to_drop)


In [13]:
sample_row = df.first()
num_columns = len(sample_row)
print("Number of columns:", num_columns)

Number of columns: 16


In [14]:
from pyspark.sql.functions import col

# Define a function to apply the renaming logic
def rename_column(name):
    return name.lower().strip().replace(' ', '_')

# Apply the renaming logic to each column
for column in df.columns:
    df = df.withColumnRenamed(column, rename_column(column))


In [15]:
df.show(1)

+--------+---------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+--------------+------------+----------------+
|app_name| category|rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|    released|last_updated|content_rating|ad_supported|in_app_purchases|
+--------+---------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+--------------+------------+----------------+
| Gakondo|Adventure|   0.0|           0|     10+|              10|              15|True|    0|     USD| 10M|Feb 26, 2020|Feb 26, 2020|      Everyone|       False|           False|
+--------+---------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+--------------+------------+----------------+
only showing top 1 row



Select and display only the columns where the count of missing values is greater than 0

In [16]:
from pyspark.sql.functions import col, count, when, isnull

# Calculate the count of missing values for each column
missing_counts = df.select([count(when(isnull(col(c)), c)).alias(c) for c in df.columns])

# Filter out columns where the count of missing values is greater than 0
columns_with_missing_values = [col_name for col_name, count in missing_counts.first().asDict().items() if count > 0]

# Select and display only the columns with missing values
df.select(*columns_with_missing_values).show()


+------+------------+--------+----------------+--------+----+------------+------------+----------------+
|rating|rating_count|installs|minimum_installs|currency|size|    released|ad_supported|in_app_purchases|
+------+------------+--------+----------------+--------+----+------------+------------+----------------+
|   0.0|           0|     10+|              10|     USD| 10M|Feb 26, 2020|       False|           False|
|   4.4|          64|  5,000+|            5000|     USD|2.9M|May 21, 2020|        True|           False|
|   0.0|           0|     50+|              50|     USD|3.7M| Aug 9, 2019|       False|           False|
|   5.0|           5|     10+|              10|     USD|1.8M|Sep 10, 2018|        True|           False|
|   0.0|           0|    100+|             100|     USD|6.2M|Feb 21, 2020|       False|           False|
|   0.0|           0|     50+|              50|     USD| 46M|Dec 24, 2018|       False|            True|
|   4.5|          12|  1,000+|            1000|     USD

 - Before going ahead, let's remove the rows with missing values in the `app_name`, `installs`, `minimum_installs`, and `currency` columns, as they are very less in number and will not affect our analysis.
 - Drop rows with missing values in specified columns ('app_name', 'installs', 'minimum_installs', 'currency')-  The inplace=True parameter modifies the DataFrame in place

In [17]:
# Specify the columns to check for missing values
columns_to_check = ['app_name', 'installs', 'minimum_installs', 'currency', 'size']

# Drop rows with missing values in the specified columns
df = df.na.drop(subset=columns_to_check)


In [18]:
from pyspark.sql.functions import col, sum as pyspark_sum

# Replace 'df' with your DataFrame
null_counts = df.select([pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Display the null counts
null_counts.show()

+--------+--------+------+------------+--------+----------------+----------------+----+-----+--------+----+--------+------------+--------------+------------+----------------+
|app_name|category|rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|released|last_updated|content_rating|ad_supported|in_app_purchases|
+--------+--------+------+------------+--------+----------------+----------------+----+-----+--------+----+--------+------------+--------------+------------+----------------+
|       0|       0| 22573|       22573|       0|               0|               0|   0|    0|       0|   0|   70758|           0|             0|           4|               2|
+--------+--------+------+------------+--------+----------------+----------------+----+-----+--------+----+--------+------------+--------------+------------+----------------+



In [19]:
# Drop duplicate rows, keeping only the first occurrence of each set of duplicated rows
df = df.dropDuplicates()


In [20]:
from pyspark.sql import functions as F

# Count the number of duplicate rows by grouping and counting
duplicate_count = df.groupBy(df.columns).count().where(F.col('count') > 1).count()

# Display the total number of duplicate rows
print("Total number of duplicate rows:", duplicate_count)



Total number of duplicate rows: 0


In [21]:
# Get summary statistics for the 'rating' column
rating_summary = df.select('rating').describe()

# Display the summary statistics
rating_summary.show()


+-------+--------------------+
|summary|              rating|
+-------+--------------------+
|  count|             2290035|
|   mean|  2.2031450149673932|
| stddev|   2.106220223728806|
|    min|             camera"|
|    max|net.jp.apps.hirot...|
+-------+--------------------+



In [22]:
from pyspark.sql import functions as F

# Calculate the average rating in each category
average_ratings = df.groupBy('category').agg(F.avg('rating').alias('average_rating'))

# Show the average ratings in each category
average_ratings.show()


+--------------------+------------------+
|            category|    average_rating|
+--------------------+------------------+
|       Music & Audio|2.2268082050884104|
|           Education| 2.246841661870555|
|              Trivia| 2.526178458379673|
|     Auto & Vehicles| 2.049612834939554|
|       Entertainment|2.4354052030558417|
|           Adventure|2.8149910036424224|
|              Arcade| 2.450480960783202|
|              Sports| 2.315538458268686|
|      Travel & Local|1.9211517115171177|
|        Food & Drink|1.2945984586476393|
|        Role Playing| 3.384498291394846|
|             Finance|2.4030697214540653|
|     Personalization|2.7587538758819026|
|              Racing|2.9602338606662095|
|               Tools| 2.346092836514766|
|com.appmk.book.AO...|              null|
|         Educational| 2.437638902079969|
|              Comics|2.8649060616802564|
|              Social|2.3989215085257616|
|    Libraries & Demo|2.0871152328334652|
+--------------------+------------

In [23]:
from pyspark.sql.functions import avg

# Calculate the average rating in each 'Category'
average_ratings = df.groupBy('category').agg(avg('rating').alias('average_rating'))

# Join the average ratings back to the original DataFrame
df_with_average_ratings = df.join(average_ratings, on='category', how='left')

# Show the DataFrame with average ratings
df_with_average_ratings.show()


+-----------------+--------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+------------------+
|         category|            app_name|rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|    released|last_updated| content_rating|ad_supported|in_app_purchases|    average_rating|
+-----------------+--------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+------------------+
|    Music & Audio|PsyRadio Alternat...|   0.0|           0|      1+|               1|               1|True|    0|     USD| 10M|Jan 25, 2021|Jan 26, 2021|Adults only 18+|        True|           False|2.2268082050884095|
|        Education|               CNJFO|   0.0|           0|     10+|              10|              28|True|    0|     U

In [24]:
# Join the DataFrame with average ratings on 'category'
df_with_avg_ratings = df.join(average_ratings, on='category', how='left')

# Replace missing values in 'rating' with the respective average ratings
df = df_with_avg_ratings.withColumn('rating', F.when(df_with_avg_ratings['rating'].isNull(), df_with_avg_ratings['average_rating']).otherwise(df_with_avg_ratings['rating'])).drop('average_rating')

# Show the updated DataFrame
df.show()


+-----------------+--------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
|         category|            app_name|rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|    released|last_updated| content_rating|ad_supported|in_app_purchases|
+-----------------+--------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
|    Music & Audio|PsyRadio Alternat...|   0.0|           0|      1+|               1|               1|True|    0|     USD| 10M|Jan 25, 2021|Jan 26, 2021|Adults only 18+|        True|           False|
|        Education|               CNJFO|   0.0|           0|     10+|              10|              28|True|    0|     USD| 10M|Mar 25, 2018|Jan 10, 2019|Adults only 18+|       False|           Fa

In [25]:
df.show(1)

+-------------+--------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
|     category|            app_name|rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|    released|last_updated| content_rating|ad_supported|in_app_purchases|
+-------------+--------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
|Music & Audio|PsyRadio Alternat...|   0.0|           0|      1+|               1|               1|True|    0|     USD| 10M|Jan 25, 2021|Jan 26, 2021|Adults only 18+|        True|           False|
+-------------+--------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
only showing to

In [26]:
from pyspark.sql import functions as F

# Calculate the average rating count in each 'Category'
average_rating_count = df.groupBy('category').agg(F.avg('rating_count').alias('average_rating_count'))

# Replace missing values in 'Rating' with the respective average ratings of their Category
df = df.join(average_rating_count, on='category', how='left') \
       .withColumn('rating_count', F.when(df['rating_count'].isNull(), average_rating_count['average_rating_count']) \
                                      .otherwise(df['rating_count'])) \
       .drop('average_rating_count')

# Show the updated DataFrame
df.show()


+---------------+-------------------------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
|       category|                             app_name|rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|    released|last_updated| content_rating|ad_supported|in_app_purchases|
+---------------+-------------------------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
|Auto & Vehicles|                            CarsNI.uk|   0.0|           0|    500+|             500|             963|True|    0|     USD|1.4M| Dec 1, 2017|Feb 24, 2018|       Everyone|        True|           False|
|      Education|                            주역정의1|   0.0|           0|    500+|             500|             784|True|    0|     USD|1.

In [27]:
# Drop rows with missing values in the 'released' column
df = df.na.drop(subset=['released'])


In [28]:
# Get the unique values of the 'installs' column
unique_installs = df.select('installs').distinct().collect()

# Convert the unique values to a list
unique_installs_list = [row['installs'] for row in unique_installs]

# Print the unique values
print(unique_installs_list)


['50,000+', '5,000,000,000+', '100+', '0', '5,000,000+', '100,000,000+', '1,000,000,000+', '1,000,000+', '500,000,000+', '10+', '4.8', '50+', '5+', '500+', '1,000+', '500,000+', '0+', '5,000+', '10,000,000+', '50,000,000+', '100,000+', '1+', '56', '10,000+', '34', '458', '4.7', '7', '0.0', '4.6', '1963', '294', '142', '50', '4.2', '64', '22', '77', '10,000,000,000+', '10', 'Maps & Navigation']


In [29]:
from pyspark.sql.functions import regexp_replace

# Remove the '+' symbol
df = df.withColumn('installs', regexp_replace('installs', '\\+', ''))

# Remove commas
df = df.withColumn('installs', regexp_replace('installs', ',', ''))

# Show the updated DataFrame
df.show()


+---------------+-------------------------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
|       category|                             app_name|rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|    released|last_updated| content_rating|ad_supported|in_app_purchases|
+---------------+-------------------------------------+------+------------+--------+----------------+----------------+----+-----+--------+----+------------+------------+---------------+------------+----------------+
|Auto & Vehicles|                            CarsNI.uk|   0.0|           0|     500|             500|             963|True|    0|     USD|1.4M| Dec 1, 2017|Feb 24, 2018|       Everyone|        True|           False|
|Auto & Vehicles|                            PKTracers|   5.0|          27|     100|             100|             181|True|    0|     US

In [30]:
from pyspark.sql.functions import when

# Replace missing values and empty strings with 0
df = df.withColumn('installs', when((df['installs'].isNull()) | (df['installs'] == ""), 0).otherwise(df['installs']))

# Convert the column to the integer type
df = df.withColumn('installs', df['installs'].cast('int'))

# Show the updated DataFrame
df.show()


+---------------+--------------------+-----------------+------------+--------+----------------+----------------+----+-----+--------+----+--------------------+-------------------+--------------+------------+----------------+
|       category|            app_name|           rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|            released|       last_updated|content_rating|ad_supported|in_app_purchases|
+---------------+--------------------+-----------------+------------+--------+----------------+----------------+----+-----+--------+----+--------------------+-------------------+--------------+------------+----------------+
|   Quick notes"| """Gom Day"" - Memo|com.migom.gomharu|       Tools|       0|               0|             10+|  10|   15|    True|   0|https://gomapps.b...|migomapps@gmail.com|  Nov 14, 2018|    Everyone|            null|
|Auto & Vehicles|           CarsNI.uk|              0.0|           0|     500|             500|         

In [31]:
# Get the unique values of the 'installs' column
unique_installs = df.select('installs').distinct().collect()

# Convert the unique values to a list
unique_installs_list = [row['installs'] for row in unique_installs]

# Print the unique values
print(unique_installs_list)


[500000, 100000, None, 1, 5000, 500, 10000, 5, 64, 50000, 5000000, 50000000, 4, 100, 1000, 1963, 1000000, 10, 50, 500000000, 1000000000, 10000000, 0, 100000000, 458, 142, 7, 77, 34, 56, 294, 22]


In [32]:
# Get the unique values of the 'size' column
unique_sizes = df.select('size').distinct().collect()

# Convert the unique values to a list
unique_sizes_list = [row['size'] for row in unique_sizes]

# Print the unique values
print(unique_sizes_list)


['8.2M', '167M', '697k', '708k', '1,023k', '163M', '970k', '275k', '622k', '8.5M', '665k', '602k', '330k', '458k', '735k', '913k', '691k', '77k', '245k', '416k', '372k', '262k', '957k', '290k', '8.6M', '10.0M', '770k', '246k', '317k', '725k', '25M', '82k', '749k', '166k', '193k', '110k', '560k', '503k', '101M', '74M', '147k', '451k', '992k', '176k', '48k', '422M', '803k', '716k', '120k', '1.5G', '1,013k', '9.2k', '521k', '979k', '34k', '1.1G', '57k', '750k', '896k', '231M', '485k', '73k', '510k', '349k', '133M', '88M', '194M', '958k', '44k', '111k', '27k', '298M', '581k', '304M', '38M', '3.0M', '880k', '565k', '997k', '314M', '15M', '234k', '607k', '391M', '228M', '178M', '197M', '919M', '78M', '574k', '519k', '955k', '788k', '951k', '8.3M', '980k', '878k', '126M', '828k', '9.6k', '821k', '50M', '4.0M', '189k', '637k', '164k', '135k', '366k', '424k', '223k', '859k', '328k', '356k', '5.9M', '6.3M', '103k', '198M', '927k', '46k', '781k', '288M', '239M', '688k', '1.8M', '34M', '66k', '184

# Observations :

1. We have varies Categories in the Content Rating Columns:
   - Everyone
   - Teen
   - Mature 17+
   - Everyone
   - Unrated
   - Adults only 18+

Now, we makes this Categories to a simple 3 Categories for better Understanding:
Everyone, teen, Adults
  - Mature 17+ ---> to Adults
  - Everyone 10+ ---> to TEEN
  - Unrated ---> to Everyone
  - Adults only 18+ ---> to Adults

In [33]:
from pyspark.sql.functions import when

# Replace "Unrated" with "Everyone"
df = df.withColumn("content_rating", when(df["content_rating"] == "Unrated", "Everyone").otherwise(df["content_rating"]))

# Replace "Everyone 10+" with "Teen"
df = df.withColumn("content_rating", when(df["content_rating"] == "Everyone 10+", "Teen").otherwise(df["content_rating"]))

# Replace "Mature 17+" with "Adults"
df = df.withColumn("content_rating", when(df["content_rating"] == "Mature 17+", "Adults").otherwise(df["content_rating"]))

# Replace "Adults only 18+" with "Adults"
df = df.withColumn("content_rating", when(df["content_rating"] == "Adults only 18+", "Adults").otherwise(df["content_rating"]))

# Show the updated DataFrame
df.show()


+---------------+--------------------+-----------------+------------+--------+----------------+----------------+----+-----+--------+----+--------------------+-------------------+--------------+------------+----------------+
|       category|            app_name|           rating|rating_count|installs|minimum_installs|maximum_installs|free|price|currency|size|            released|       last_updated|content_rating|ad_supported|in_app_purchases|
+---------------+--------------------+-----------------+------------+--------+----------------+----------------+----+-----+--------+----+--------------------+-------------------+--------------+------------+----------------+
|   Quick notes"| """Gom Day"" - Memo|com.migom.gomharu|       Tools|       0|               0|             10+|  10|   15|    True|   0|https://gomapps.b...|migomapps@gmail.com|  Nov 14, 2018|    Everyone|            null|
|Auto & Vehicles|           CarsNI.uk|              0.0|           0|     500|             500|         

In [34]:
# Get the unique values of the 'content_rating' column
unique_content_ratings = df.select('content_rating').distinct().collect()

# Convert the unique values to a list
unique_content_ratings_list = [row['content_rating'] for row in unique_content_ratings]

# Print the unique values
print(unique_content_ratings_list)


['Apr 16, 2019', 'Teen', 'May 08, 2019', 'bok@yanosik.pl', 'Apr 06, 2021', 'May 19, 2021', 'Dec 18, 2019', 'Mar 05, 2018', 'May 22, 2019', 'Everyone', 'May 21, 2020', 'Nov 02, 2020', 'Adults', 'Sep 23, 2019', 'Dec 25, 2020', 'Oct 22, 2016', 'Sep 24, 2017', 'Dec 14, 2019', 'Feb 25, 2021', 'Apr 12, 2019', 'Apr 11, 2019', 'Mar 22, 2021', 'Dec 24, 2019', 'May 16, 2018', 'Jun 24, 2020', 'Nov 14, 2018', 'Sep 26, 2018', 'Jun 27, 2017', 'May 09, 2018', 'Dec 19, 2019', 'Dec 20, 2019', 'May 09, 2019', 'Jul 15, 2018', 'May 21, 2017', 'Jan 5, 2021', 'May 20, 2021', 'Jan 25, 2020', 'Sep 30, 2018', 'May 10, 2017', 'Jun 14, 2021', 'Jul 31, 2017', 'Sep 16, 2020', 'Apr 20, 2021', 'Jul 15, 2020']


In [35]:
from pyspark.sql.functions import when

# Create a new column 'type' based on the value of the 'free' column
df = df.withColumn('type', when(df['free'] == True, 'Free').otherwise('Paid'))

# Drop the 'free' column
df = df.drop('free')

# Show the updated DataFrame
df.show()


+---------------+--------------------+-----------------+------------+--------+----------------+----------------+-----+--------+----+--------------------+-------------------+--------------+------------+----------------+----+
|       category|            app_name|           rating|rating_count|installs|minimum_installs|maximum_installs|price|currency|size|            released|       last_updated|content_rating|ad_supported|in_app_purchases|type|
+---------------+--------------------+-----------------+------------+--------+----------------+----------------+-----+--------+----+--------------------+-------------------+--------------+------------+----------------+----+
|   Quick notes"| """Gom Day"" - Memo|com.migom.gomharu|       Tools|       0|               0|             10+|   15|    True|   0|https://gomapps.b...|migomapps@gmail.com|  Nov 14, 2018|    Everyone|            null|Paid|
|Auto & Vehicles|           CarsNI.uk|              0.0|           0|     500|             500|         

In [36]:
# Get the unique values of the 'rating' column
unique_ratings = df.select('rating').distinct().collect()

# Convert the unique values to a list
unique_ratings_list = [row['rating'] for row in unique_ratings]

# Print the unique values
print(unique_ratings_list)


['3.1201711319960714', '1.0', '2.6', '3.1', '2.1225496382298483', '4.2', 'Entertainment', '2.8789000849823982', ' camera"', '2.7060391425908645', '4.4', '2.7', '3.8', '1.7', '1.4390768776205152', '2.9', '4.5', '2.4835674025364036', '2.4', '2.5', '4.9', '2.398921508525762', '3.4', '1.6', '1.8369481457697867', '0.0', '2.4232731137088206', '2.2248140276301784', '2.662008206613565', '1.9303731254564438', '3.3', '1.8', '3.210887167752153', 'com.menshchyna.android', 'com.app.aziuyr', '4.3', '3.5', '2.2268082050884095', '4.8', '2.3155384582686853', '4.1', '2.450480960783204', '1.9211517115171188', '1.4', '2.2468416618705507', 'Educational', '3.0851603281133473', '4.6', '5.0', '2.519764878765612', 'Social', '1.485380653354971', 'Shopping', 'Productivity', '4.0', '1.9', '1.3', 'com.andromo.dev807396.app1066982', '1.6237962637868204', '2.4252091115803154', '3.6', '1.5', '2.8', '2.049612834939554', '1.8480025244556608', '3.2796822244289974', 'Business', '2.0', '2.9602338606662086', '3.2', '2.2', 

In [37]:
from pyspark.sql import functions as F

# Calculate the maximum value in the 'rating_count' column
max_rating_count = df.agg(F.max('rating_count')).collect()[0][0]

# Print the maximum value
print("Maximum rating count:", max_rating_count)


Maximum rating count: pl.neptis.yanosik.mobi.android


In [38]:
from pyspark.sql.functions import when

# Create a new column 'rating_type' with default value 'NoRatingProvided'
df = df.withColumn('rating_type', F.lit('NoRatingProvided'))

# Update 'rating_type' based on conditions
df = df.withColumn('rating_type',
                   when((df['rating_count'] > 0) & (df['rating_count'] <= 10000.0), 'Less than 10k')
                   .when((df['rating_count'] > 10000) & (df['rating_count'] <= 500000.0), 'Between 10k and 500k')
                   .when((df['rating_count'] > 500000) & (df['rating_count'] <= 138557570.0), 'More than 500k')
                   .otherwise(df['rating_type']))

# Group by 'rating_type' and count occurrences
rating_type_counts = df.groupBy('rating_type').count().orderBy('count', ascending=False)

# Show the counts
rating_type_counts.show()


+--------------------+-------+
|         rating_type|  count|
+--------------------+-------+
|       Less than 10k|1167478|
|    NoRatingProvided|1038318|
|Between 10k and 500k|  34464|
|      More than 500k|   1590|
+--------------------+-------+



In [39]:
# Select the 'rating_type' column
rating_type_column = df.select('rating_type')

# Show the values of the 'rating_type' column
rating_type_column.show()


+----------------+
|     rating_type|
+----------------+
|NoRatingProvided|
|NoRatingProvided|
|NoRatingProvided|
|   Less than 10k|
|   Less than 10k|
|NoRatingProvided|
|NoRatingProvided|
|NoRatingProvided|
|NoRatingProvided|
|NoRatingProvided|
|   Less than 10k|
|   Less than 10k|
|NoRatingProvided|
|   Less than 10k|
|   Less than 10k|
|NoRatingProvided|
|   Less than 10k|
|   Less than 10k|
|   Less than 10k|
|NoRatingProvided|
+----------------+
only showing top 20 rows



In [40]:
# Drop the 'rating_type' column
df = df.drop('rating_type')

# Show the updated DataFrame
df.show()


+---------------+--------------------+-----------------+------------+--------+----------------+----------------+-----+--------+------+--------------------+-------------------+--------------+------------+----------------+----+
|       category|            app_name|           rating|rating_count|installs|minimum_installs|maximum_installs|price|currency|  size|            released|       last_updated|content_rating|ad_supported|in_app_purchases|type|
+---------------+--------------------+-----------------+------------+--------+----------------+----------------+-----+--------+------+--------------------+-------------------+--------------+------------+----------------+----+
|   Quick notes"| """Gom Day"" - Memo|com.migom.gomharu|       Tools|       0|               0|             10+|   15|    True|     0|https://gomapps.b...|migomapps@gmail.com|  Nov 14, 2018|    Everyone|            null|Paid|
|Auto & Vehicles| Parque Movil DipCas|              0.0|           0|     100|             100| 

In [41]:
# printing null values in the table 
from pyspark.sql.functions import col, sum as pyspark_sum

# Replace 'df' with your DataFrame
null_counts = df.select([pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Display the null counts
null_counts.show()

+--------+--------+------+------------+--------+----------------+----------------+-----+--------+----+--------+------------+--------------+------------+----------------+----+
|category|app_name|rating|rating_count|installs|minimum_installs|maximum_installs|price|currency|size|released|last_updated|content_rating|ad_supported|in_app_purchases|type|
+--------+--------+------+------------+--------+----------------+----------------+-----+--------+----+--------+------------+--------------+------------+----------------+----+
|       0|       0|     0|           0|      13|               0|               0|    0|       0|   0|       0|           0|             0|           4|               2|   0|
+--------+--------+------+------------+--------+----------------+----------------+-----+--------+----+--------+------------+--------------+------------+----------------+----+



In [43]:
# Drop rows with null values
df = df.na.drop()

# Show the cleaned DataFrame
df.show(5) 

+---------------+--------------------+------+------------+--------+----------------+----------------+-----+--------+------+------------+------------+--------------+------------+----------------+----+
|       category|            app_name|rating|rating_count|installs|minimum_installs|maximum_installs|price|currency|  size|    released|last_updated|content_rating|ad_supported|in_app_purchases|type|
+---------------+--------------------+------+------------+--------+----------------+----------------+-----+--------+------+------------+------------+--------------+------------+----------------+----+
|Auto & Vehicles| Parque Movil DipCas|   0.0|           0|     100|             100|             108|    0|     USD|1,017k| Jun 5, 2018|May 27, 2021|      Everyone|       False|           False|Free|
|Auto & Vehicles|      Misfat Catalog|   4.5|          36|    5000|            5000|            6408|    0|     USD|  1.0M|Jun 12, 2015|Jul 05, 2015|      Everyone|       False|           False|Free|


In [44]:
# printing null values in the table 
from pyspark.sql.functions import col, sum as pyspark_sum

# Replace 'df' with your DataFrame
null_counts = df.select([pyspark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

# Display the null counts
null_counts.show()

+--------+--------+------+------------+--------+----------------+----------------+-----+--------+----+--------+------------+--------------+------------+----------------+----+
|category|app_name|rating|rating_count|installs|minimum_installs|maximum_installs|price|currency|size|released|last_updated|content_rating|ad_supported|in_app_purchases|type|
+--------+--------+------+------------+--------+----------------+----------------+-----+--------+----+--------+------------+--------------+------------+----------------+----+
|       0|       0|     0|           0|       0|               0|               0|    0|       0|   0|       0|           0|             0|           0|               0|   0|
+--------+--------+------+------------+--------+----------------+----------------+-----+--------+----+--------+------------+--------------+------------+----------------+----+



In [45]:
# Count the number of elements in the RDD
num_rows = df.count()
print("Number of rows",num_rows)
# count the number of columns 
sample_row = df.first()
num_columns = len(sample_row)
print("Number of columns:", num_columns)

Number of rows 2241831
Number of columns: 16


In [46]:
# Save the DataFrame as a single CSV file
df.coalesce(1).write.csv("D:/DBDA Project/new_google.csv", header=True)

In [49]:
spark.stop()